Before you submit this notebook, make sure everything runs as expected in the local test cases. 
Please, paste the solution to the designed cell and do not change anything else.

Also, please, leave your first and last names below

In [3]:
FirstName = "Aleksey"
LastName = "Ryabykin"

---

In [4]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.preprocessing import OneHotEncoder

In [5]:
import numpy as np
from sklearn.base import BaseEstimator


def entropy(y):  
    """
    Computes entropy of the provided distribution. Use log(value + eps) for numerical stability
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, n_classes)
        One-hot representation of class labels for corresponding subset
    
    Returns
    -------
    float
        Entropy of the provided subset
    """
    EPS = 0.0005

    # YOUR CODE HERE
    prob = np.mean(y, axis=0)
    return -(prob * np.log(prob + EPS)).sum()
    
def gini(y):
    """
    Computes the Gini impurity of the provided distribution
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, n_classes)
        One-hot representation of class labels for corresponding subset
    
    Returns
    -------
    float
        Gini impurity of the provided subset
    """

    # YOUR CODE HERE
    prob = np.mean(y, axis=0)
    return 1 - (prob**2).sum()
    
def variance(y):
    """
    Computes the variance the provided target values subset
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, 1)
        Target values vector
    
    Returns
    -------
    float
        Variance of the provided target vector
    """
    
    # YOUR CODE HERE
    
    return y.var()

def mad_median(y):
    """
    Computes the mean absolute deviation from the median in the
    provided target values subset
    
    Parameters
    ----------
    y : np.array of type float with shape (n_objects, 1)
        Target values vector
    
    Returns
    -------
    float
        Mean absolute deviation from the median in the provided vector
    """

    # YOUR CODE HERE
    
    return np.mean(np.abs(y - np.median(y)))


def one_hot_encode(n_classes, y):
    y_one_hot = np.zeros((len(y), n_classes), dtype=float)
    y_one_hot[np.arange(len(y)), y.astype(int)[:, 0]] = 1.
    return y_one_hot


def one_hot_decode(y_one_hot):
    return y_one_hot.argmax(axis=1)[:, None]


class Node:
    """
    This class is provided "as is" and it is not mandatory to it use in your code.
    """
    def __init__(self, feature_index, threshold, proba=0):
        self.feature_index = feature_index
        self.value = threshold
        self.proba = proba
        self.left_child = None
        self.right_child = None
        
        
class DecisionTree(BaseEstimator):
    all_criterions = {
        'gini': (gini, True), # (criterion, classification flag)
        'entropy': (entropy, True),
        'variance': (variance, False),
        'mad_median': (mad_median, False)
    }

    def __init__(self, n_classes=None, max_depth=np.inf, min_samples_split=2, 
                 criterion_name='gini', debug=False):

        assert criterion_name in self.all_criterions.keys(), 'Criterion name must be on of the following: {}'.format(self.all_criterions.keys())
        
        self.n_classes = n_classes
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion_name = criterion_name

        self.depth = 0
        self.root = None # Use the Node class to initialize it later
        self.debug = debug

        
        
    def make_split(self, feature_index, threshold, X_subset, y_subset):
        """
        Makes split of the provided data subset and target values using provided feature and threshold
        
        Parameters
        ----------
        feature_index : int
            Index of feature to make split with

        threshold : float
            Threshold value to perform split

        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset

        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels for corresponding subset
        
        Returns
        -------
        (X_left, y_left) : tuple of np.arrays of same type as input X_subset and y_subset
            Part of the providev subset where selected feature x^j < threshold
        (X_right, y_right) : tuple of np.arrays of same type as input X_subset and y_subset
            Part of the providev subset where selected feature x^j >= threshold
        """

        # YOUR CODE HERE
        mask = X_subset[:, feature_index] < threshold
        X_left, X_right = X_subset[mask], X_subset[~mask]
        y_left, y_right = y_subset[mask], y_subset[~mask]
        return (X_left, y_left), (X_right, y_right)
    
    def make_split_only_y(self, feature_index, threshold, X_subset, y_subset):
        """
        Split only target values into two subsets with specified feature and threshold
        
        Parameters
        ----------
        feature_index : int
            Index of feature to make split with

        threshold : float
            Threshold value to perform split

        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset

        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels for corresponding subset
        
        Returns
        -------
        y_left : np.array of type float with shape (n_objects_left, n_classes) in classification 
                   (n_objects, 1) in regression 
            Part of the provided subset where selected feature x^j < threshold

        y_right : np.array of type float with shape (n_objects_right, n_classes) in classification 
                   (n_objects, 1) in regression 
            Part of the provided subset where selected feature x^j >= threshold
        """

        # YOUR CODE HERE
        mask = X_subset[:, feature_index] < threshold
        y_left, y_right = y_subset[mask], y_subset[~mask]
        return y_left, y_right

    def choose_best_split(self, X_subset, y_subset):
        """
        Greedily select the best feature and best threshold w.r.t. selected criterion
        
        Parameters
        ----------
        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset

        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels or target values for corresponding subset
        
        Returns
        -------
        feature_index : int
            Index of feature to make split with

        threshold : float
            Threshold value to perform split

        """
        # YOUR CODE HERE

        threshold = None
        feature_index = 0
        Q_min = np.inf

        for column in range(X_subset.shape[1]):
            for threshold_ in np.unique(X_subset[:, column]):
                y_left, y_right = \
                    self.make_split_only_y(feature_index=column,
                                           threshold=threshold_,
                                           X_subset=X_subset,
                                           y_subset=y_subset)
                Q = (y_left.shape[0] / y_subset.shape[0]) * self.criterion(y_left) + \
                    (y_right.shape[0] / y_subset.shape[0]) * self.criterion(y_right)
                
                if Q < Q_min:
                    threshold = threshold_
                    feature_index = column
                    Q_min = Q
        return feature_index, threshold
    
    def make_tree(self, X_subset, y_subset, depth=0):
        """
        Recursively builds the tree
        
        Parameters
        ----------
        X_subset : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the selected subset

        y_subset : np.array of type float with shape (n_objects, n_classes) in classification 
                   (n_objects, 1) in regression 
            One-hot representation of class labels or target values for corresponding subset
        
        Returns
        -------
        root_node : Node class instance
            Node of the root of the fitted tree
        """

        # YOUR CODE HERE
        if X_subset.shape[0] > self.min_samples_split and depth < self.max_depth:
            new_node = Node(*self.choose_best_split(X_subset, y_subset))
            (X_left, y_left), (X_right, y_right) = self.make_split(new_node.feature_index, new_node.value, X_subset, y_subset)
            new_node.left_child, new_node.right_child = self.make_tree(X_left, y_left, depth + 1), self.make_tree(X_right, y_right, depth + 1)
            return new_node
        else:
            return y_subset.mean(axis=0)
        
    def fit(self, X, y):
        """
        Fit the model from scratch using the provided data
        
        Parameters
        ----------
        X : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the data to train on

        y : np.array of type int with shape (n_objects, 1) in classification 
                   of type float with shape (n_objects, 1) in regression 
            Column vector of class labels in classification or target values in regression
        
        """
        assert len(y.shape) == 2 and len(y) == len(X), 'Wrong y shape'
        self.criterion, self.classification = self.all_criterions[self.criterion_name]
        if self.classification:
            if self.n_classes is None:
                self.n_classes = len(np.unique(y))
            y = one_hot_encode(self.n_classes, y)

        self.root = self.make_tree(X, y)
    
    def predict(self, X):
        """
        Predict the target value or class label  the model from scratch using the provided data
        
        Parameters
        ----------
        X : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the data the predictions should be provided for

        Returns
        -------
        y_predicted : np.array of type int with shape (n_objects, 1) in classification 
                   (n_objects, 1) in regression 
            Column vector of class labels in classification or target values in regression
        
        """

        # YOUR CODE HERE
        y_predicted = np.zeros(X.shape[0])

        for i in range(X.shape[0]):
                current_node = self.root

                while type(current_node) is Node:
                    if X[i, current_node.feature_index] < current_node.value:
                        current_node = current_node.left_child
                    else:
                        current_node = current_node.right_child

                if self.all_criterions[self.criterion_name][1]:
                    y_predicted[i] = np.argmax(current_node)
                else:
                    y_predicted[i] = current_node
        return y_predicted
        
    def predict_proba(self, X):
        """
        Only for classification
        Predict the class probabilities using the provided data
        
        Parameters
        ----------
        X : np.array of type float with shape (n_objects, n_features)
            Feature matrix representing the data the predictions should be provided for

        Returns
        -------
        y_predicted_probs : np.array of type float with shape (n_objects, n_classes)
            Probabilities of each class for the provided objects
        
        """
        assert self.classification, 'Available only for classification problem'

        # YOUR CODE HERE
        
        y_predicted_probs = np.zeros((X.shape[0], self.n_classes))

        for i in range(X.shape[0]):
                current_node = self.root

                while type(current_node) is Node:
                    if X[i, current_node.feature_index] < current_node.value:
                        current_node = current_node.left_child
                    else:
                        current_node = current_node.right_child

                y_predicted_probs[i] = current_node
        
        return y_predicted_probs

### Test 0: Initialization (0.01 points)

In [6]:
# do not change this cell
import numpy as np
import unittest
import sys
import io

from sklearn.datasets import fetch_california_housing, load_digits
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

digits_data = load_digits(n_class=2).data
digits_target = load_digits(n_class=2).target[:, None]



### Test 1: Make splits loops (0.1 points)

In [7]:
X = np.ones((4, 5), dtype=float) * np.arange(4)[:, None]
y = np.arange(4)[:, None] + np.asarray([0.2, -0.3, 0.1, 0.4])[:, None]
class_estimator = DecisionTree(max_depth=5, criterion_name='gini')

(X_l, y_l), (X_r, y_r) = class_estimator.make_split(1, 1., X, y)

flag_X = np.array_equal(X[:1], X_l) and np.array_equal(X[1:], X_r) 
flag_y = np.array_equal(y[:1], y_l) and np.array_equal(y[1:], y_r)
assert flag_X and flag_y

### Test 2: Gini accuracy (0.2 points)

In [8]:
class_estimator = DecisionTree(max_depth=5, criterion_name='gini')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
accuracy_gini = accuracy_score(y_test, ans)
assert 0.96 < accuracy_gini

NameError: name 'X_train' is not defined

### Test 3: Entropy accuracy (0.2 points)

In [ ]:
class_estimator = DecisionTree(max_depth=5, criterion_name='entropy')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
accuracy = accuracy_score(y_test, ans)
assert 0.96 < accuracy

### Test 4: Entropy probabilities (0.2 points)

In [ ]:
class_estimator = DecisionTree(max_depth=10, criterion_name='entropy')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
reference = np.array([0.48611111, 0.51388889])
assert np.abs(np.sum(class_estimator.predict_proba(X_test).mean(axis=0) - reference)) < 1e-6

### Test 5: MSE mad_median (0.15 points)

In [9]:
housing = fetch_california_housing()
random_indices = np.random.choice(np.arange(len(housing.data)), 500)

regr_data = housing.data[random_indices]
regr_target = housing.target[random_indices, None]
RX_train, RX_test, Ry_train, Ry_test = train_test_split(regr_data, regr_target, test_size=0.2, random_state=42)

regressor = DecisionTree(max_depth=8, criterion_name='mad_median')
regressor.fit(RX_train, Ry_train)
predictions_mad = regressor.predict(RX_test)
mse = mean_squared_error(Ry_test, predictions_mad)
assert 0.4 < mse < 2

c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Test 6: MSE Variance (0.15 points)

In [10]:
housing = fetch_california_housing()
random_indices = np.random.choice(np.arange(len(housing.data)), 500)

regr_data = housing.data[random_indices]
regr_target = housing.target[random_indices, None]
RX_train, RX_test, Ry_train, Ry_test = train_test_split(regr_data, regr_target, test_size=0.2, random_state=42)

regressor = DecisionTree(max_depth=8, criterion_name='variance')
regressor.fit(RX_train, Ry_train)
predictions_mad = regressor.predict(RX_test)
mse = mean_squared_error(Ry_test, predictions_mad)
assert 0.5 < mse < 1.8

C:\Users\Aleksey Ryabykin\AppData\Local\Temp\ipykernel_13560\1688245858.py:57: RuntimeWarning: Degrees of freedom <= 0 for slice
  return y.var()
c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
